# `functools`

The `functools` module is for higher-order functions: functions that act on or return other functions. In general, any callable object can be treated as a function for the purposes of this module.

Common functions in functools are as follows

* lru_cache
* partial
* reduce

## `lru_cache`

It is a decorator which provides caching of the functions as shown below.

**Syntax**
```python
functools.lru_cache(maxsize=64, typed=False)
```

**Example:**

In [12]:
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [13]:
print(fibonacci(11))

89


In [14]:
%%timeit

for i in range(10):
    fibonacci(i)

1.97 µs ± 24 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Same function call without caching 

In [15]:
def fibonacci_x(n):
    if n < 2:
        return n
    else:
        return fibonacci_x(n - 1) + fibonacci_x(n - 2)

In [16]:
%%timeit

for i in range(10):
    fibonacci_x(i)

51.6 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


As a dictionary is used to cache results thus the function arguments must be hashable. Thus below code will always fail.
- **What is hashable**: All of Python’s immutable built-in objects are hashable, while no mutable containers (such as lists or dictionaries) are. Objects which are instances of user-defined classes are hashable by default; they all compare unequal, and their hash value is their id()

```python
@lru_cache(maxsize=128)
def set_list(lst, no):
    lst.append(no)
    return lst

lst = [1]
lst = set_list(lst, 1)
lst = set_list(lst, 1)
print(lst)
```

**Output:**

```python
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-10-bfa4246327b8> in <module>()
      5 
      6 lst = [1]
----> 7 lst = set_list(lst, 1)
      8 lst = set_list(lst, 1)
      9 print(lst)

TypeError: unhashable type: 'list'

```


> <center>**!! NOTE !!**</center> 
> - If maxsize is set to None, the LRU feature is disabled and the cache can grow without bound. The LRU feature performs best when maxsize is a power-of-two.
> - If typed is set to true, function arguments of different types will be cached separately. For example, f(3) and f(3.0) will be treated as distinct calls with distinct results.

### When NOT to use

- When function is not pure function
- Your function will only run once and its not recursive in nature 

### `cache_clear()` & `cache_info()`

It can be used to clear the cache of your function and `cache_info` provides the info of the cache

In [23]:
fibonacci.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [24]:
from functools import lru_cache
fibonacci.cache_clear()

In [26]:
fibonacci.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

## `total_ordering`

In python 3.x, class function `__cmp__` is no longer entertained, thus developers have to create all the "rich comparison” methods, such as, `__eq__()`, `__lt__()`, `__le__()`, `__gt__()`, or `__ge__()`, where 
- `__eq__()`: Function to evaluate the object is equal to other instance
- `__lt__()`: Function to evaluate the object is less than other instance 
- `__le__()`: Function to evaluate the object is less or equal to other instance
- `__gt__()`: Function to evaluate the object is greater than other instance
- `__ge__()`: Function to evaluate the object is greater than or equal to other instance
- `__ne__()`: Function to evaluate the object is not equal to other instance

`total_ordering` class decorator helps in supplying other "rich comparision" methods, if one of the `__lt__()`, `__le__()`, `__gt__()`, or `__ge__()` method in addition of `__eq__()` is defined, then python compiler will provide the others as shown in the below example.

In [30]:
from functools import total_ordering

@total_ordering
class Mangos(object):
    def __init__(self, code_num):
        self.code_num = code_num
    
    def __eq__(self, other):
        return self.code_num == other.code_num
    
    def __lt__(self, other):
        return other.code_num > self.code_num
    
a = Mangos(3)
b = Mangos(3)
c = Mangos(2)
d = Mangos(5)

In [10]:
print("~^"*3, "equal to")
print(a == b)
print(a == c)
print(a == d)

True
True
True


In [46]:
print("~^"*3, "not equal to")
print(a != b)
print(a != c)
print(a != d)

~^~^~^ not equal to
False
True
True


In [44]:
print("~^"*3, "greater than")
print(a > b)
print(a > c)
print(a > d)
print("~^"*3, "greater than or equal to")
print(a >= b)
print(a >= c)
print(a >= d)

~^~^~^ greater than
False
True
False
~^~^~^ greater than or equal to
True
True
False


In [43]:
print("~^"*3, "Less than")
print(a < b)
print(a < c)
print(a < d)
print("~^"*3, "Less than or equal to")
print(a <= b)
print(a <= c)
print(a <= d)

~^~^~^ Less than
False
False
True
~^~^~^ Less than or equal to
True
False
True


## partial

`functools.partial` does the follows:

* Makes a new version of a function with one or more arguments already filled in.
* New version of a function documents itself.

In [47]:
def power(base, exponent):
    return base ** exponent

In [48]:
def square(base):
    return power(base, 2)

def cube(base):
    return power(base, 3)

Now lets see the magic of partial

In [68]:
from functools import partial

square = partial(power, exponent=2)
cube = partial(power, exponent=3)

print(square(2))
print(cube(2))

print(square(2, exponent=4))
print(cube(2, exponent=9))

4
8
16
512


In [73]:
from functools import partial

def multiply(x,y):
        return x * y

# create a new function that multiplies by 2
db2 = partial(multiply,2)
print(db2(4))
db4 = partial(multiply, 4)
print(db4(3))

8
12


In [47]:
from functools import partial
 
#----------------------------------------------------------------------
def add(x, y):
    """"""
    return x + y
 
#----------------------------------------------------------------------
def multiply(x, y):
    """"""
    return x * y
 
#----------------------------------------------------------------------
def run(func):
    """"""
    print (func())
 
#----------------------------------------------------------------------
def main():
    """"""
    a1 = partial(add, 1, 2)
    m1 = partial(multiply, 5, 8)
    run(a1)
    run(m1)
 
if __name__ == "__main__":
    main()

3
40


```python
# Reading a very large text file.
CHUNK_SIZE = 2048
with open('files/a_vedic_reader_for_students.txt') as f:
    for chunk in iter(partial(f.read, CHUNK_SIZE), ''):
        print(chunk)
```

## `wraps`

## TODO ##

In [101]:
def another_function(func):
    """
    A function that accepts another function
    """
 
    def wrapper():
        """
        A wrapping function
        """
        val = "The result of %s is %s" % (func(),
                                          eval(func())
                                          )
        return val
    return wrapper
 
#----------------------------------------------------------------------
@another_function
def a_function():
    """A pretty useless function"""
    return "1+1"
 
#----------------------------------------------------------------------
if __name__ == "__main__":
    print (a_function.__name__)
    print (a_function.__doc__)
    print(a_function())

wrapper

        A wrapping function
        
The result of 1+1 is 2


In [81]:
from functools import wraps
 
#----------------------------------------------------------------------
def another_function(func):
    """
    A function that accepts another function
    """
 
    @wraps(func)
    def wrapper():
        """
        A wrapping function
        """
        val = "The result of %s is %s" % (func(),
                                          eval(func())
                                          )
        return val
    return wrapper
 
#----------------------------------------------------------------------
@another_function
def a_function():
    """A pretty useless function"""
    return "1+1"
 
#----------------------------------------------------------------------
if __name__ == "__main__":
    #a_function()
    print (a_function.__name__)
    print (a_function.__doc__)
    print(a_function())

a_function
A pretty useless function
The result of 1+1 is 2


Here we import wraps from the functools module and use it as a decorator for the nested wrapper function inside of another_function to map the __name__ and __doc__ to the wrapper function

### update_wrapper

The partial object does not have __name__ or __doc__ attributes by default, and without those attributes decorated functions are more difficult to debug. Using update_wrapper(), copies or adds attributes from the original function to the partial object.

In [1]:
import functools


def myfunc1(a, b=2):
    print ('\tcalled myfunc1 with:', (a, b))
    return

def myfunc(a, b=2):
    """Docstring for myfunc()."""
    print ('\tcalled myfunc with:', (a, b))
    return

def show_details(name, f):
    """Show details of a callable object."""
    print ('%s:' % name)
    print ('\tobject:', f)
    print ('\t__name__:',) 
    try:
        print (f.__name__)
    except AttributeError:
        print ('(no __name__)')
    print ('\t__doc__', repr(f.__doc__))
    print
    return


show_details('myfunc1', myfunc1)
print("~"*20)
show_details('myfunc', myfunc)

p1 = functools.partial(myfunc, b=4)
print("+"*20)
show_details('raw wrapper', p1)
print("^"*20)
print ('Updating wrapper:')
print ('\tassign:', functools.WRAPPER_ASSIGNMENTS)
print ('\tupdate:', functools.WRAPPER_UPDATES)
print("*"*20)

functools.update_wrapper(p1, myfunc)
show_details('updated wrapper', p1)

myfunc1:
	object: <function myfunc1 at 0x7f4dc83b4ea0>
	__name__:
myfunc1
	__doc__ None
~~~~~~~~~~~~~~~~~~~~
myfunc:
	object: <function myfunc at 0x7f4dcffedbf8>
	__name__:
myfunc
	__doc__ 'Docstring for myfunc().'
++++++++++++++++++++
raw wrapper:
	object: functools.partial(<function myfunc at 0x7f4dcffedbf8>, b=4)
	__name__:
(no __name__)
	__doc__ 'partial(func, *args, **keywords) - new function with partial application\n    of the given arguments and keywords.\n'
^^^^^^^^^^^^^^^^^^^^
Updating wrapper:
	assign: ('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
	update: ('__dict__',)
********************
updated wrapper:
	object: functools.partial(<function myfunc at 0x7f4dcffedbf8>, b=4)
	__name__:
myfunc
	__doc__ 'Docstring for myfunc().'
